In [137]:
import pandas as pd
import pyodbc, os
from ydata_profiling import ProfileReport

In [138]:
sales_conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=..\GO-databases DEDS-week 4 & 5\go_sales.accdb;')
crm_conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=..\GO-databases DEDS-week 4 & 5\go_crm.accdb;')

In [139]:
product = pd.read_sql("SELECT * FROM product", sales_conn)
order_header = pd.read_sql("SELECT * FROM order_header", sales_conn)
order_details = pd.read_sql("SELECT * FROM order_details", sales_conn)
retailer = pd.read_sql("SELECT * FROM retailer", crm_conn)
retailer_site = pd.read_sql("SELECT * FROM crm_retailer_site", crm_conn)




C:\Users\Wouter\AppData\Local\Temp\ipykernel_19736\2903736725.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product = pd.read_sql("SELECT * FROM product", sales_conn)
C:\Users\Wouter\AppData\Local\Temp\ipykernel_19736\2903736725.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  order_header = pd.read_sql("SELECT * FROM order_header", sales_conn)
C:\Users\Wouter\AppData\Local\Temp\ipykernel_19736\2903736725.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  order_details = pd.read_sql("SELECT * FROM order_details", 

Given code

In [140]:
numeric_columns = ['PRODUCT_NUMBER', 'PRODUCT_TYPE_CODE', 'PRODUCTION_COST', 'MARGIN']
non_numeric_columns = list(product.drop(numeric_columns, axis = 1).columns)
non_numeric_columns

['INTRODUCTION_DATE',
 'PRODUCT_IMAGE',
 'LANGUAGE',
 'PRODUCT_NAME',
 'DESCRIPTION']

In [141]:
product['INTRODUCTION_DATE'] = product['INTRODUCTION_DATE'].fillna('onbekend')
product['INTRODUCTION_DATE'].isna().sum()

0

In [142]:
modus = product['LANGUAGE'].mode()[0]
product['LANGUAGE'] = product['LANGUAGE'].fillna(modus)
product['LANGUAGE'].isna().sum()


0

In [143]:
subset_product_name_contains_granite = product.loc[(~product['PRODUCT_NAME'].isna()) & (product['PRODUCT_NAME'].str.contains('Granite')), :]
subset_product_name_contains_granite = product.loc[(~product['PRODUCT_NAME'].isna()) & (product['PRODUCT_NAME'].str.contains('Granite')), :]
product.loc[(~product['PRODUCT_NAME'].isna()) & (product['PRODUCT_NAME'].str.contains('Granite')), 'PRODUCT_TYPE_CODE'] = subset_product_name_contains_granite['PRODUCT_TYPE_CODE'].fillna(9)

In [144]:
product['INTRODUCTION_YEAR'] = product['INTRODUCTION_DATE'].str[-4:]
subset_introduction_year_2011_2019 = product.loc[(product['INTRODUCTION_YEAR'] == '2011') | (product['INTRODUCTION_YEAR'] == '2019'), :]
grouped_subset_introduction_year_2011_2019 = subset_introduction_year_2011_2019.groupby(['INTRODUCTION_YEAR', 'PRODUCT_TYPE_CODE'])['PRODUCT_NUMBER'].count().reset_index()
grouped_subset_introduction_year_2011_2019 = grouped_subset_introduction_year_2011_2019.rename(columns = {'PRODUCT_NUMBER' : 'Aantal producten'})
grouped_subset_introduction_year_2011_2019.sort_values(['INTRODUCTION_YEAR', 'Aantal producten'], ascending = [True, False])
product.loc[(product['INTRODUCTION_YEAR'] == '2011'), 'PRODUCT_TYPE_CODE'] = product.loc[(product['INTRODUCTION_YEAR'] == '2011'), 'PRODUCT_TYPE_CODE'].fillna(1)
product.loc[(product['INTRODUCTION_YEAR'] == '2019'), 'PRODUCT_TYPE_CODE'] = product.loc[(product['INTRODUCTION_YEAR'] == '2019'), 'PRODUCT_TYPE_CODE'].fillna(19)
product['PRODUCT_TYPE_CODE'].isna().sum()

0

## My Code

### Product Categorisch

 Remove all unused columns

In [145]:
product = product.drop('INTRODUCTION_DATE', axis='columns')
product = product.drop('INTRODUCTION_YEAR', axis='columns')
# product = product.drop('PRODUCTION_COST', axis='columns')
# product = product.drop('MARGIN', axis='columns')
product = product.drop('PRODUCT_IMAGE', axis='columns')
product = product.drop('PRODUCT_NAME', axis='columns')
product = product.drop('PRODUCT_TYPE_CODE', axis='columns')
product = product.drop('DESCRIPTION', axis='columns')   

product

,PRODUCT_NUMBER,PRODUCTION_COST,MARGIN,LANGUAGE
0,53,28.55,.28,EN
1,54,35.33,.33,EN
2,55,4,.5,EN
3,56,9,.5,EN
4,57,40,.5,EN
...,...,...,...,...
110,108,472.22,.43,EN
111,109,35,.43,EN
112,110,60,.5,EN
113,111,81.8,P111GE5PT20.jpg,EN


Assign correct types

In [146]:

# product['PRODUCT_NUMBER'] = product['PRODUCT_NUMBER'].astype(int)
# product['PRODUCTION_COST'] = product['PRODUCTION_COST'].astype(float)
# product['MARGIN'] = product['MARGIN'].astype(float)
# product['LANGUAGE'] = product['LANGUAGE'].astype(str)

### Order Details Categorisch

In [147]:
order_details = order_details.drop('UNIT_COST', axis='columns')

## Order Header Categorisch

Remove unused data

In [148]:
order_header = order_header.drop('ORDER_METHOD_CODE', axis='columns')
order_header = order_header.drop('SALES_BRANCH_CODE', axis='columns')
order_header = order_header.drop('SALES_STAFF_CODE', axis='columns')

In [149]:
order_header

,ORDER_NUMBER,RETAILER_NAME,RETAILER_SITE_CODE,RETAILER_CONTACT_CODE,ORDER_DATE
0,1831,Sport & Freizeit,36,52,2022-11-11
1,1832,Sport & Freizeit,38,43,2022-11-19
2,1833,Sport & Freizeit,40,60,2022-11-11
3,1834,Die Zeltstadt,47,40,2022-11-15
4,1835,Die Zeltstadt,48,57,2022-12-14
...,...,...,...,...,...
5355,9238,Fu Chin Sport Equipment Co.,362,353,2021-10-17
5356,9239,"Chen Yu Enterprise Co.,",365,356,2022-02-16
5357,9240,"Chen Yu Enterprise Co.,",365,356,2022-10-12
5358,9241,"Chen Yu Enterprise Co.,",363,354,2022-01-20


Fill in unknown/impossible to know data

In [150]:

order_header['RETAILER_NAME'].isna().sum()
order_header['RETAILER_NAME'] = order_header['RETAILER_NAME'].fillna('Unknown')
order_header['RETAILER_NAME'].isna().sum()

order_header['ORDER_DATE'].isna().sum()
order_header['ORDER_DATE'] = order_header['ORDER_DATE'].fillna('Unknown')
order_header['ORDER_DATE'].isna().sum()


0

Buuranalyse Retailer site/contact codes

In [151]:
order_header.loc[order_header['RETAILER_SITE_CODE'].isna(), :]



order_header['RETAILER_SITE_CODE'].isna().sum()
Retailer_Names = order_header['RETAILER_NAME'].unique()
for retailer in Retailer_Names:
    subset = order_header.loc[order_header['RETAILER_NAME'].str.contains(retailer), :]
    if subset['RETAILER_SITE_CODE'].mode().size > 0:
        mode = subset['RETAILER_SITE_CODE'].mode()[0]
        order_header.loc[(order_header['RETAILER_NAME'].str.contains(retailer)), 'RETAILER_SITE_CODE'] = order_header.loc[(order_header['RETAILER_NAME'] == retailer), 'RETAILER_SITE_CODE'].fillna(mode)

order_header.loc[order_header['RETAILER_SITE_CODE'].isna(), :]

# Find the most used codes for a specific retailer
# subset = order_header.loc[order_header['RETAILER_NAME'].str.contains('Sport & Freizeit'), :]
# mode = subset['RETAILER_SITE_CODE'].mode()[0]
# mode
# order_header['RETAILER_CONTACT_CODE'].isna().sum()


,ORDER_NUMBER,RETAILER_NAME,RETAILER_SITE_CODE,RETAILER_CONTACT_CODE,ORDER_DATE
128,1974,37,NaN,Holstein Golf,2020-10-22 00:00:00
357,2218,190,NaN,Altitudes extrÛmes,2020-03-22 00:00:00
432,2302,186,NaN,Cordages Discount,2021-08-11 00:00:00
1076,1415,17,NaN,Falcon Outfitters,2020-07-23 00:00:00
1102,1442,77,NaN,Maximum Sports,2022-07-31 00:00:00
1156,1498,77,NaN,Maximum Sports,2021-01-20 00:00:00
1171,1513,75,NaN,Maximum Sports,2022-02-27 00:00:00
1208,1552,76,NaN,Maximum Sports,2021-12-24 00:00:00
1419,1782,1/7/2020,NaN,60,Unknown
1506,5890,150,NaN,Hartman's,2020-06-13 00:00:00


# Rapportages

In [152]:

# profile = ProfileReport(product, title = 'Kwaliteitsrapportage product')
# try:
#     os.remove("product.html")
# except:
#     print("tu tu tu tu tututu")
    
# profile.to_file("product.html")
product

,PRODUCT_NUMBER,PRODUCTION_COST,MARGIN,LANGUAGE
0,53,28.55,.28,EN
1,54,35.33,.33,EN
2,55,4,.5,EN
3,56,9,.5,EN
4,57,40,.5,EN
...,...,...,...,...
110,108,472.22,.43,EN
111,109,35,.43,EN
112,110,60,.5,EN
113,111,81.8,P111GE5PT20.jpg,EN


In [153]:
# This takes like 30 mins & the html won't be able to open in your lifetime
# profile = ProfileReport(order_details, title = 'Kwaliteitsrapportage product', dark_mode= True)
# try:
#     os.remove("order_details.html")
# except:
#     print("tu tu tu tu tututu")
    
# profile.to_file("order_details.html")
order_details

,ORDER_DETAIL_CODE,ORDER_NUMBER,PRODUCT_NUMBER,QUANTITY,UNIT_PRICE,UNIT_SALE_PRICE
0,78.37,1258,17,92.0,84.41,48616
1,48617,1352,None,190.0,84.41,67.08
2,48618,1544,17,82.0,84.41,78.37
3,48619,1249,17,190.0,84.41,72.26
4,48620,1268,17,152.0,84.41,78.05
...,...,...,...,...,...,...
43058,102951,8614,11,16.0,350,325
43059,102952,8611,11,18.0,350,350
43060,102953,8591,11,18.0,350,325
43061,102954,8619,11,18.0,350,350
